In [69]:
%rm -rf stonks/

In [70]:
## Retrieve csv source data from 'AnalyticsArmory' github folder
## Source data will be imported in /data/ subfolder
%%capture
!git clone https://github.com/IS737StockPicker/stonks.git

In [71]:
# import required packages
%matplotlib inline
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.neural_network import MLPClassifier #this is the neural network part
import matplotlib.pylab as plt
from sklearn.metrics import accuracy_score

In [72]:
# Special package from the class book to import regression summary statistics
!pip install -U dmba;
from dmba import regressionSummary

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [73]:
# load the data
stocks_df = pd.read_csv('/content/stonks/master_data.csv')
stocks_df

,Date,XLE,XLF,XLU,XLI,XLK,XLV,XLY,IYR,AAPL,...,Huff_headline_min,Huff_body_min,NYT_headline_mean,NYT_headline_median,NYT_body_mean,NYT_body_median,NYT_headline_max,NYT_headline_min,NYT_body_max,NYT_body_min
0,2022-03-14,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-0.80,-0.67,-0.05,0.0,-0.03,0.00,0.88,-0.73,0.91,-0.96
1,2022-03-15,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,...,-0.76,-0.85,-0.09,0.0,0.02,0.00,0.84,-0.86,0.88,-0.91
2,2022-03-16,0.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,...,-0.25,-0.56,-0.06,0.0,-0.00,0.00,0.86,-0.80,0.88,-0.90
3,2022-03-17,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,-0.90,-0.49,-0.02,0.0,-0.05,0.00,0.61,-0.84,0.82,-0.89
4,2022-03-18,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,...,-0.60,-0.76,-0.06,0.0,-0.05,-0.08,0.83,-0.82,0.86,-0.97
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
120,2022-09-07,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,-0.78,-0.57,0.09,0.0,-0.07,0.00,0.65,-0.86,0.84,-0.93
121,2022-09-08,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,...,-0.77,-0.83,-0.04,0.0,-0.02,0.00,0.75,-0.90,0.92,-0.93
122,2022-09-09,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,...,-0.60,0.00,-0.01,0.0,0.14,0.14,0.81,-0.80,0.85,-0.91
123,2022-09-12,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,-0.79,0.00,-0.10,0.0,-0.06,-0.13,0.75,-0.90,0.89,-0.89


Specifying which columns in the dataframe are our target variables

In [74]:
tickers = stocks_df.iloc[:,:15]
tickers = tickers.drop(columns=['Date'])
tickers

,XLE,XLF,XLU,XLI,XLK,XLV,XLY,IYR,AAPL,XLB,XLP,SPY,^DJI,NDX
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0
2,0.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0
3,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
4,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
120,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
121,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0
122,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
123,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0


Baseline Accuracy Scores

In [75]:
baseline_acc = {}
for ticker in tickers:
  baseline_acc[ticker] = round(sum(stocks_df[ticker])/125,2)
  print(sum(stocks_df[ticker])/125)

0.544
0.536
0.52
0.592
0.544
0.568
0.504
0.504
0.568
0.48
0.544
0.568
0.536
0.52


Neural Net Classification algorithm run for each of the target variables using 6 iterations of input data (mean/median/min_max for headline sentiment score and body sentiment score)

In [102]:
present_acc_scores={}
for ticker in tickers:
  #print(ticker)
  mean_headline_var = [ticker,'guardian_headline_mean','Huff_headline_mean','NYT_headline_mean']
  median_body_var = [ticker,'guardian_body_median','Huff_body_median','NYT_body_median']
  mean_body_var = [ticker,'guardian_body_mean','Huff_body_mean','NYT_body_mean']
  median_headline_var = [ticker,'guardian_headline_median','Huff_headline_median','NYT_headline_median']
  min_max_headline_var = [ticker,'guardian_headline_min','guardian_headline_max','Huff_headline_min','Huff_headline_max','NYT_headline_min','NYT_headline_max']
  min_max_body_var = [ticker,'guardian_body_min','guardian_body_max','Huff_body_min','Huff_body_max','NYT_body_min','NYT_body_max']
  model_inputs = [mean_headline_var,mean_body_var,median_headline_var,median_body_var,min_max_headline_var,min_max_body_var]
  input_arr = [baseline_acc[ticker]]
  for inputs in range(len(model_inputs)):
    accuracy_df = stocks_df[model_inputs[inputs]]
    y_nonscaled = accuracy_df[[ticker]]
    X_nonscaled = accuracy_df.drop(columns=[ticker])
    scaleOutput = MinMaxScaler()
    scaleInput = MinMaxScaler()
    y = scaleOutput.fit_transform(y_nonscaled)
    X = scaleInput.fit_transform(X_nonscaled)
    X_train = X[:90]
    y_train = y[:90]
    X_valid = X[35:]
    y_valid = y[35:]
    stock_nnet = MLPClassifier(
      hidden_layer_sizes=(2), 
      activation='logistic', 
      solver='lbfgs', 
      random_state=1)
    stock_nnet.fit(X_train, y_train.ravel())
    y_pred = stock_nnet.predict(X_valid)
    input_arr.append(round(accuracy_score(y_valid, y_pred),2))
    
  present_acc_scores[ticker] = input_arr

/usr/local/lib/python3.9/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:541: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)
/usr/local/lib/python3.9/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:541: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)
/usr/local/lib/python3.9/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:541: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REAC

In [103]:
present_outcomes_df = pd.DataFrame.from_dict(present_acc_scores).T
present_outcomes_df.columns = ['baseline_acc', 'headline_mean_acc', 'body_mean_acc', 'headline_median_acc', 'body_median_acc', 'headline_min_max', 'body_min_max']
present_outcomes_df

,baseline_acc,headline_mean_acc,body_mean_acc,headline_median_acc,body_median_acc,headline_min_max,body_min_max
XLE,0.54,0.62,0.56,0.59,0.62,0.68,0.59
XLF,0.54,0.58,0.60,0.64,0.48,0.63,0.63
XLU,0.52,0.58,0.53,0.52,0.51,0.66,0.60
XLI,0.59,0.56,0.69,0.67,0.59,0.67,0.64
XLK,0.54,0.62,0.63,0.62,0.62,0.60,0.67
XLV,0.57,0.58,0.63,0.59,0.62,0.63,0.70
XLY,0.50,0.58,0.47,0.52,0.57,0.60,0.61
IYR,0.50,0.61,0.58,0.61,0.60,0.58,0.60
AAPL,0.57,0.53,0.64,0.64,0.62,0.72,0.60
XLB,0.48,0.62,0.56,0.58,0.52,0.54,0.59


Writing dataframe to CSV

In [84]:
acc_df.to_csv("neural_net_present.csv")

Importing stock data shifted up by one day in an attempt to predict the next day stock price given previous days input data

In [85]:
future_df = pd.read_csv('/content/stonks/master_data_future.csv')
future_df

,Date,XLE,XLF,XLU,XLI,XLK,XLV,XLY,IYR,AAPL,...,Huff_headline_min,Huff_body_min,NYT_headline_mean,NYT_headline_median,NYT_body_mean,NYT_body_median,NYT_headline_max,NYT_headline_min,NYT_body_max,NYT_body_min
0,2022-03-14,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,...,-0.80,-0.67,-0.05,0.0,-0.03,0.00,0.88,-0.73,0.91,-0.96
1,2022-03-15,0.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,...,-0.76,-0.85,-0.09,0.0,0.02,0.00,0.84,-0.86,0.88,-0.91
2,2022-03-16,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,-0.25,-0.56,-0.06,0.0,-0.00,0.00,0.86,-0.80,0.88,-0.90
3,2022-03-17,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,...,-0.90,-0.49,-0.02,0.0,-0.05,0.00,0.61,-0.84,0.82,-0.89
4,2022-03-18,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,...,-0.60,-0.76,-0.06,0.0,-0.05,-0.08,0.83,-0.82,0.86,-0.97
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
120,2022-09-07,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,...,-0.78,-0.57,0.09,0.0,-0.07,0.00,0.65,-0.86,0.84,-0.93
121,2022-09-08,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,...,-0.77,-0.83,-0.04,0.0,-0.02,0.00,0.75,-0.90,0.92,-0.93
122,2022-09-09,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,-0.60,0.00,-0.01,0.0,0.14,0.14,0.81,-0.80,0.85,-0.91
123,2022-09-12,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-0.79,0.00,-0.10,0.0,-0.06,-0.13,0.75,-0.90,0.89,-0.89


Baseline accuracy scores for dataset 

In [100]:
future_baseline_acc = {}
for ticker in tickers:
  future_baseline_acc[ticker] = round(sum(future_df[ticker])/125,2)
print(future_baseline_acc)

{'XLE': 0.56, 'XLF': 0.54, 'XLU': 0.54, 'XLI': 0.59, 'XLK': 0.54, 'XLV': 0.57, 'XLY': 0.51, 'IYR': 0.5, 'AAPL': 0.58, 'XLB': 0.48, 'XLP': 0.54, 'SPY': 0.58, '^DJI': 0.54, 'NDX': 0.53}


Neural Net Classification algorithm run for each of the target ticker variables using 6 iterations of input data (mean/median/min_max for headline sentiment score and body sentiment score)

In [98]:
future_acc_scores={}
for ticker in tickers:
  #print(ticker)
  mean_headline_var = [ticker,'guardian_headline_mean','Huff_headline_mean','NYT_headline_mean']
  median_body_var = [ticker,'guardian_body_median','Huff_body_median','NYT_body_median']
  mean_body_var = [ticker,'guardian_body_mean','Huff_body_mean','NYT_body_mean']
  median_headline_var = [ticker,'guardian_headline_median','Huff_headline_median','NYT_headline_median']
  min_max_headline_var = [ticker,'guardian_headline_min','guardian_headline_max','Huff_headline_min','Huff_headline_max','NYT_headline_min','NYT_headline_max']
  min_max_body_var = [ticker,'guardian_body_min','guardian_body_max','Huff_body_min','Huff_body_max','NYT_body_min','NYT_body_max']
  model_inputs = [mean_headline_var,mean_body_var,median_headline_var,median_body_var,min_max_headline_var,min_max_body_var]
  input_arr = [future_baseline_acc[ticker]]
  for inputs in range(len(model_inputs)):
    accuracy_df = future_df[model_inputs[inputs]]
    y_nonscaled = accuracy_df[[ticker]]
    X_nonscaled = accuracy_df.drop(columns=[ticker])
    scaleOutput = MinMaxScaler()
    scaleInput = MinMaxScaler()
    y = scaleOutput.fit_transform(y_nonscaled)
    X = scaleInput.fit_transform(X_nonscaled)
    X_train = X[:90]
    y_train = y[:90]
    X_valid = X[35:]
    y_valid = y[35:]
    stock_nnet = MLPClassifier(
      hidden_layer_sizes=(2), 
      activation='logistic', 
      solver='lbfgs', 
      random_state=1)
    stock_nnet.fit(X_train, y_train.ravel())
    y_pred = stock_nnet.predict(X_valid)
    input_arr.append(round(accuracy_score(y_valid, y_pred),2))
    
  future_acc_scores[ticker] = input_arr


/usr/local/lib/python3.9/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:541: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)
/usr/local/lib/python3.9/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:541: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)
/usr/local/lib/python3.9/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:541: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REAC

Converting result from previous code block into df

In [101]:
future_outcomes_df = pd.DataFrame.from_dict(future_acc_scores).T
future_outcomes_df.columns = ['baseline_acc', 'headline_mean_acc', 'body_mean_acc', 'headline_median_acc', 'body_median_acc', 'headline_min_max', 'body_min_max']
future_outcomes_df


,baseline_acc,headline_mean_acc,body_mean_acc,headline_median_acc,body_median_acc,headline_min_max,body_min_max
XLE,0.56,0.66,0.67,0.67,0.66,0.60,0.53
XLF,0.54,0.59,0.58,0.53,0.43,0.51,0.60
XLU,0.54,0.61,0.60,0.48,0.59,0.60,0.66
XLI,0.59,0.61,0.60,0.60,0.60,0.66,0.50
XLK,0.54,0.57,0.60,0.59,0.49,0.61,0.57
XLV,0.57,0.59,0.60,0.70,0.58,0.67,0.63
XLY,0.51,0.59,0.53,0.61,0.56,0.61,0.56
IYR,0.50,0.57,0.56,0.53,0.60,0.66,0.68
AAPL,0.58,0.67,0.58,0.59,0.62,0.67,0.61
XLB,0.48,0.63,0.56,0.54,0.56,0.59,0.52
